In [1]:
from os import listdir
import pickle
import numpy as np

def pickle_read(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
        return data

        


In [2]:
graph = listdir("graph")

graph_data = []
graph_label = []

for dir_path in graph:
    for data_path in listdir("graph" + '/' + dir_path):
        datas = pickle_read("graph" + '/' + dir_path + '/' + data_path)

        for data in datas:
            graph_data.append(np.array([data[0]]))
            graph_label.append(data[1])


for i in range(10):

    print(graph_data[i].shape)
    print(graph_label[i])

(1, 40, 100)
0
(1, 40, 100)
1
(1, 40, 100)
0
(1, 40, 100)
0
(1, 40, 100)
0
(1, 40, 100)
0
(1, 40, 100)
0
(1, 40, 100)
0
(1, 40, 100)
1
(1, 40, 100)
1


In [3]:
import torch
from torch.utils.data import DataLoader, TensorDataset


# 转换为PyTorch张量
graph_data_tensor = torch.tensor(np.array(graph_data), dtype=torch.float32)
graph_label_tensor = torch.tensor(np.array(graph_label), dtype=torch.long)

# 创建TensorDataset和DataLoader
dataset = TensorDataset(graph_data_tensor, graph_label_tensor)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

for x, y in data_loader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([32, 1, 40, 100])
torch.Size([32])


In [4]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 10 * 25, 256)
        self.fc2 = nn.Linear(256, 2)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [5]:
import torch.optim as optim

use_gpu = torch.cuda.is_available()

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)

if (use_gpu):
    model = model.to(device="cuda", dtype=torch.float32)
    criterion = criterion.to(device="cuda", dtype=torch.float32)

In [6]:

        
        
min_loss = 1e9
for epoch in range(100):  # 进行10个训练周期
    running_loss = 0.0
    correct = 0
    total = 0

    
    
    for inputs, labels in data_loader:
        if (use_gpu):
            inputs, labels = inputs.to(device="cuda", dtype=torch.float32), labels.to(device="cuda", dtype=torch.int64)
        
        optimizer.zero_grad()  # 清空梯度
        outputs = model(inputs)  # 前向传播
        loss = criterion(outputs, labels)  # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权重
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)  # 获取预测结果
        total += labels.size(0)  # 更新总样本数
        correct += (predicted == labels).sum().item()  # 更新正确预测的样本数

    
    epoch_loss = running_loss / len(data_loader)
    epoch_acc = 100 * correct / total  # 计算准确率

    if (epoch_loss < min_loss):
        
        torch.save(model.state_dict() ,"checkpoint_cls_1_9.pth" )
        print(f"save model, old_min_loss:{min_loss}, new_min_loss:{epoch_loss}")
        min_loss = epoch_loss

    print(f"Epoch {epoch+1}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

save model, old_min_loss:1000000000.0, new_min_loss:0.6783379613594851
Epoch 1, Loss: 0.6783, Accuracy: 57.90%
save model, old_min_loss:0.6783379613594851, new_min_loss:0.5967198617889075
Epoch 2, Loss: 0.5967, Accuracy: 67.24%
save model, old_min_loss:0.5967198617889075, new_min_loss:0.4873046944007922
Epoch 3, Loss: 0.4873, Accuracy: 75.50%
save model, old_min_loss:0.4873046944007922, new_min_loss:0.3699857457979338
Epoch 4, Loss: 0.3700, Accuracy: 82.88%
save model, old_min_loss:0.3699857457979338, new_min_loss:0.2641927601423579
Epoch 5, Loss: 0.2642, Accuracy: 88.26%
save model, old_min_loss:0.2641927601423579, new_min_loss:0.17628411231112237
Epoch 6, Loss: 0.1763, Accuracy: 93.11%
save model, old_min_loss:0.17628411231112237, new_min_loss:0.11041720238995309
Epoch 7, Loss: 0.1104, Accuracy: 95.98%
save model, old_min_loss:0.11041720238995309, new_min_loss:0.070343507765425
Epoch 8, Loss: 0.0703, Accuracy: 97.61%
save model, old_min_loss:0.070343507765425, new_min_loss:0.04793525